# TP11 Statistique exploratoire spatiale

## Consigne du TP

### Etape 1 : Préparation du raster des évébements
* Importer les points;
* Filtrer par rapport au pays MALI;
* Filtrer par rappor à l'année 2020;
* Rasteriser les points (résolution spatiale = 5000 m);
* Binariser le raster des événements obtenu (1 si >= 5; 0 sinon).

### Etape 2 : Préparation du raster population Mali
* Importer le raster worldpop_mali pour l'année 2020;
* Diminuer la résolution spatiale en faisant la somme ( 100 m -> 5000 m );
* Binariser le raster worldpop_5000m (1 si >= 50; 0 sinon).

### Etape 3 : Calcul du confliction diffusion indicator (CDI)
* Multiplier les deux rasters;
* Calculer par admin le nombre de 1 avec le raster multiplié (a);
* Faire la même chose pour le raster Worldpop binarisé (b);
* l'indicateur est donné par a/b.

## Membre du groupe
* Mame Balla BOUSSO
* Ameth FAYE
* Hiledegarde EDIMA BIYENDA
* Papa Amadou NIANG


# Importation des packages necessaires

In [2]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import rasterio
from rasterio.windows import Window
from affine import Affine
from rasterio.transform import from_origin
from rasterio.features import rasterize
from skimage.measure import block_reduce
from rasterstats import zonal_stats

# Etape 1 : Préparation du raster des évébements

## Importation des points

In [3]:
# Définir le chemin du dossier
dossier_path ='D:\Statistique exploratoire spatiale\Cours2\Statistique-Exploratoire-Spatiale\TP11\Groupe - Python'
os.chdir(dossier_path)

# Construire le chemin complet vers le fichier CSV
points = 'data\Points_data.csv'

# Charger les données
df = pd.read_csv(points)

# Filtrer les données pour le Mali en 2020
df_mali_2020 = df[(df['country'] == 'Mali') & (df['year'] == 2020)]

# Créer le GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_mali_2020,
    geometry=gpd.points_from_xy(df_mali_2020['longitude'], df_mali_2020['latitude']),
    crs='EPSG:4326'
)

# Sélectionner les colonnes qui nous interesse
gdf = gdf[['event_id_cnty', 'year', 'latitude', 'longitude', 'geometry']]

# Afficher les premières lignes
print(gdf.head())

      event_id_cnty  year  latitude  longitude                 geometry
42789       MLI4682  2020   14.3493    -3.6102  POINT (-3.6102 14.3493)
42794       MLI4683  2020   14.7983    -1.3025  POINT (-1.3025 14.7983)
42802       MLI4684  2020   14.8059    -6.0182  POINT (-6.0182 14.8059)
42804       MLI4616  2020   13.9581    -3.7103  POINT (-3.7103 13.9581)
42832       MLI4681  2020   16.1118    -0.3408  POINT (-0.3408 16.1118)


## Rasteriser les événements

In [5]:
# Rasteriser les événements
minx, miny, maxx, maxy = gdf.total_bounds
width = int((maxx - minx) / 5000) + 1
height = int((maxy - miny) / 5000) + 1
transform = from_origin(minx, maxy, 5000, 5000)

raster_data = np.zeros((height, width), dtype=int)

event_counts = rasterize(
    [(geom, 1) for geom in gdf.geometry],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    all_touched=True,
    dtype='int32',
    merge_alg=rasterio.enums.MergeAlg.add
)

meta = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': 'int32',
    'crs': gdf.crs,
    'transform': transform
}

with rasterio.open(r'outputs\raster_evenements_2020.tif', 'w', **meta) as dst:
    dst.write(event_counts, 1)

## Binariser le raster des événements

In [6]:
# Binariser le raster des événements
binary_events = np.where(event_counts >= 5, 1, 0)
with rasterio.open(r'outputs\raster_evenements_binaire.tif', 'w', **meta) as dst:
    dst.write(binary_events.astype('int8'), 1)

# Etape 2 : Préparation du raster population Mali

In [6]:
# Chemins d'accès aux fichiers
worldpop_path = r'data\worldpop_mali.tif'
worldpop2_path = r'outputs\worldpop_mali_5000m.tif'

# Taille du bloc d'agrégation
block_size = 50

with rasterio.open(worldpop_path) as src:
    # Lire les métadonnées du raster source
    src_meta = src.meta.copy()
    src_transform = src.transform
    src_crs = src.crs
    src_width = src.width
    src_height = src.height

    # Calculer les dimensions du raster agrégé
    out_width = src_width // block_size
    out_height = src_height // block_size

    # Gérer les bords si les dimensions ne sont pas multiples de block_size
    if src_width % block_size != 0:
        out_width += 1
    if src_height % block_size != 0:
        out_height += 1

    # Mettre à jour la transformation pour le raster agrégé
    new_transform = src_transform * Affine.scale(block_size, block_size)

    # Mettre à jour les métadonnées pour le raster de sortie
    out_meta = src_meta.copy()
    out_meta.update({
        'height': out_height,
        'width': out_width,
        'transform': new_transform,
        'dtype': 'int8',
        'count': 1,
        'nodata': -1  # Valeur nodata compatible avec int8
    })

    # Créer le fichier raster de sortie
    with rasterio.open(worldpop2_path, 'w', **out_meta) as dst:
        # Parcourir chaque bloc
        for row in range(out_height):
            for col in range(out_width):
                # Définir la fenêtre de lecture
                window = Window(col * block_size, row * block_size, block_size, block_size)
                
                # Lire les données de la fenêtre
                data = src.read(1, window=window)
                
                # Vérifier si la fenêtre dépasse les limites du raster
                if data.shape[0] != block_size or data.shape[1] != block_size:
                    pad_height = block_size - data.shape[0]
                    pad_width = block_size - data.shape[1]
                    data = np.pad(data, ((0, pad_height), (0, pad_width)), 'constant', constant_values=0)
                
                # Calculer la somme des populations dans le bloc
                sum_pop = np.sum(data)
                
                # Binariser selon le seuil (ex: 50)
                binary = 1 if sum_pop >= 50 else 0
                
                # Écrire la valeur binaire dans le raster de sortie
                dst.write(np.array([[binary]], dtype='int8'), 1, window=Window(col, row, 1, 1))

print("Agrégation et binarisation terminées avec succès.")

Agrégation et binarisation terminées avec succès.


# Etape 3 : Calcul du confliction diffusion indicator (CDI)

In [16]:
# Charger le raster binaire WorldPop
with rasterio.open(worldpop2_path) as src:
    binary_worldpop = src.read(1)
    meta = src.meta.copy()

with rasterio.open(r'outputs\raster_multiplied.tif', 'w', **meta) as dst:
    multiplied_raster = binary_events * binary_worldpop
    dst.write(multiplied_raster.astype('int8'), 1)

# Charger les unités administratives
admin_shp = 'data\mli_adm_ab_shp\mli_admbnda_adm1_1m_gov_20211220.shp'
gdf_admin = gpd.read_file(admin_shp)
gdf_admin = gdf_admin.to_crs(gdf.crs)

# Calculer les statistiques zonales
stats_multiplied = zonal_stats(
    gdf_admin,
    r'outputs\raster_multiplied.tif',
    stats='sum',
    nodata=0
)
gdf_admin['sum_multiplied'] = [s['sum'] for s in stats_multiplied]

stats_wp = zonal_stats(
    gdf_admin,
    worldpop2_path,
    stats='sum',
    nodata=0
)
gdf_admin['sum_wp'] = [s['sum'] for s in stats_wp]

# Calculer l'indicateur en évitant la division par zéro
gdf_admin['indicateur_diffusion'] = gdf_admin.apply(
    lambda row: row['sum_multiplied'] / row['sum_wp'] if row['sum_wp'] > 0 else 0,
    axis=1
)


# Sauvegarder les résultats
gdf_admin.to_file(r'outputs\admin_mali_indicateur_diffusion.shp')

C:\Users\user\AppData\Local\Temp\ipykernel_5752\2591909992.py:41: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_admin.to_file(r'outputs\admin_mali_indicateur_diffusion.shp')
C:\Users\user\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field date create as date field, though DateTime requested.
  ogr_write(
C:\Users\user\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field validOn create as date field, though DateTime requested.
  ogr_write(
C:\Users\user\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'sum_multiplied' to 'sum_multip'
  ogr_write(
C:\Users\user\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'indicateur_diffusion' to 'indicateur'
  ogr_write(
